In [ ]:
from pyspark.sql.session import SparkSession
import pyspark.sql.functions as f

from pyspark.sql.types import StructField, StructType, StringType, IntegerType, FloatType


In [ ]:
spark = (( 
            SparkSession
                .builder
                .appName('medium_blog')
                # .master('Local[*]')
                .getOrCreate()
            ))

In [ ]:
dataframeteste = spark.range(0, 100)
dataframeteste.show()

print(type(dataframeteste))

In [ ]:
dtframe = (( 
            spark
            .createDataFrame([
                ('A', 30),
                ('B', 40),
                ('C', 60),
                ('D', 90),
                ('A', 20),
                ('C', 40)],
            ['Product', 'Price'])
                             
))

((
    dtframe
        .select('price')
        .where('price > 40')
)).show()

In [ ]:
#creating dataframe with schema

productschema = ((StructType(
                    [StructField('product', StringType(), True),
                    StructField('price', FloatType(), True),
                    StructField('category', StringType(), True)])
                ))

productschema_sql = "product STRING, price FLOAT, category STRING"

productframe = ((spark.createDataFrame([('rice', 3.99, 'food'),
                                        ('whey protein', 9.99, 'supplements'),
                                        ('chicken', 4.99, 'food'),
                                        ('beans', 2.99, 'food'),
                                        ('water', 1.50, 'drink'),
                                        ('soda', 2.99, 'drink')
                                    ],schema = productschema)
                ))

dfProduct = ((spark.createDataFrame(
                                    [('rice', 3.99, 'food'),
                                        ('whey protein', 9.99, 'supplements'),
                                        ('chicken', 4.99, 'food'),
                                        ('beans', 2.99, 'food'),
                                        ('water', 1.50, 'drink'),
                                        ('soda', 2.99, 'drink')],
                                    schema = productschema_sql
                                    )
            ))

# productframe.show(1)
dfProduct.show(1)

In [48]:
sumbycategory = productframe.groupBy('category').agg({'price': 'avg'})
sumbycategory.show()

totalcategory = productframe.selectExpr("cast(sum(price) as decimal(12,2)) totalbycategory")
totalcategory.show()

total = ((productframe
            .select('category', 'price')
            .groupBy('category')
            .agg({'price':'sum'})
            .filter("category = food")
          ))
total.show()

+-----------+------------------+
|   category|        avg(price)|
+-----------+------------------+
|       food|3.9899999300638833|
|supplements| 9.989999771118164|
|      drink|2.2450000047683716|
+-----------+------------------+

+---------------+
|totalbycategory|
+---------------+
|          26.45|
+---------------+



AnalysisException: [UNRESOLVED_COLUMN.WITH_SUGGESTION] A column or function parameter with name `food` cannot be resolved. Did you mean one of the following? [`category`, `sum(price)`].; line 1 pos 12;
'Filter (category#195 = 'food)
+- Aggregate [category#195], [category#195, sum(price#194) AS sum(price)#1205]
   +- Project [category#195, price#194]
      +- LogicalRDD [product#193, price#194, category#195], false
